In [ ]:
"""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import pandas as pd
#from sqlalchemy import create_engine

# Create list of dictionaries for each row.
listings = []
scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
scrape_path = scrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   

# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
df = df[df.city != "Belleview, FL"]
df = df[df.city != "High Springs, FL"]
df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
times = (df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
times = times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
df = df.drop('times', axis=1)
df = pd.concat([df, times], axis=1, join_axes=[df.index])
df['streetaddy'] = df.addy.str.split(',').str[0]
df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df['cityaddy'] = df.addy.str.split(',').str[1]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('addy', axis=1)

# Ocala market
oc6_df = df.loc[df['theater'] == 'Ocala Center 6']
ocd_dr = df.loc[df['theater'] == 'Ocala Drive-In']
bv_df = df.loc[df['theater'] == 'Belleview Cinemas']
mt_df = df.loc[df['theater'] == 'Marion Theatre']
omp_df = df.loc[df['theater'] == 'Old Mill Playhouse']
regaloca_df = df.loc[df['theater'] == 'Regal Hollywood Stadium 16 & IMAX - Ocala']
rialto_df = df.loc[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']
amc_df = df.loc[df['theater'] == 'AMC Lake Square 12']
barn_df = df.loc[df['theater'] == 'Barnstorm Theater']

# Gainesville market
celeb_df = df.loc[df['theater'] == 'Celebration Pointe 10']
hipp_df = df.loc[df['theater'] == 'Hippodrome State Theatre']
butler_df = df.loc[df['theater'] == 'Regal Butler Town Center 14']
regalgnv_df = df.loc[df['theater'] == 'Regal Royal Park Stadium 16']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']
starke_df = df.loc[df['theater'] == 'Florida Twin']


In [ ]:
oc6_df

In [ ]:
import csv
from datetime import datetime

scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
newfile = scrapedir + scrapefile

def ocadaily_pn(newfile):
    """ Builds print narrative for Ocala daily editions """
    global listings
    listing = []
    listings = []
    ocala_market = 'Ocala, FL'
    gnv_market = ['Gainesville, FL', 'Starke, FL']
    weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
    with open(newfile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headers = next(csv_reader)
        for row in csv_reader:
            if row[9] in gnv_market:
                row[0] = row[0].split(', FL')[0]
                row[5] = row[5].replace('p | ', ' p.m., ')
                row[5] = row[5].replace('a | ', ' a.m., ')
                row[5] = row[5].replace('0p', '0 p.m.')
                row[5] = row[5].replace('5p', '5 p.m.')
                row[5] = row[5].replace(':00', '')
                row[8] = datetime.strptime(row[8], '%Y-%m-%d').weekday()
                movieday = weekDays[row[8]]
                movie_stuff = f"\"{row[2]}\", {row[3]}, {row[4]}, {row[6]}, " + movieday + f" at {row[5]}\n"
                theater_stuff = f'{row[1]}, {row[0]}, '
                listing = theater_stuff + movie_stuff
                listings.append(listing)
    
    return listings

if __name__ == '__main__':
    ocadaily_pn(newfile)
    print(listings)

In [ ]:
import csv
from datetime import datetime

scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-13.csv'
newfile = scrapedir + scrapefile

def ocadaily_pn(newfile):
    """ Builds print narrative for Ocala daily editions """
    global listings
    listing = []
    listings = []
    ocala_market = 'Ocala, FL'
    gnv_market = ['Gainesville, FL', 'Starke, FL']
    weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
    with open(newfile) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headers = next(csv_reader)
        for row in csv_reader:
            if row[9] in gnv_market:
                row[0] = row[0].split(', FL')[0]
                row[5] = row[5].replace('p | ', ' p.m., ')
                row[5] = row[5].replace('a | ', ' a.m., ')
                row[5] = row[5].replace('0p', '0 p.m.')
                row[5] = row[5].replace('5p', '5 p.m.')
                row[5] = row[5].replace(':00', '')
                row[8] = datetime.strptime(row[8], '%Y-%m-%d').weekday()
                movieday = weekDays[row[8]]
                movie_stuff = f"\"{row[2]}\", {row[3]}, {row[4]}, {row[6]}, " + movieday + f" at {row[5]}\n"
                theater_stuff = f'{row[1]}, {row[0]}, '
                listing = theater_stuff + movie_stuff
                listings.append(listing)
    
    return listings

if __name__ == '__main__':
    ocadaily_pn(newfile)
    print(listings)

In [19]:
import csv
from datetime import datetime
from more_itertools import unique_everseen
from operator import itemgetter
import os

# access new data scrape
scrapedir = '/Users/Doug/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-06-20.csv'
newfile = scrapedir + scrapefile

# remove duplicate rows from overlap of markets scraped
with open(newfile, 'r') as f, open('outfile.csv', 'w') as out_file:
    out_file.writelines(unique_everseen(f))

out_file = 'outfile.csv'
listing = []
listings = []
weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
with open(out_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    headers = next(csv_reader)
    for row in csv_reader:
        row[0] = row[0].split(', FL')[0]
        row[5] = row[5].replace('p | ', ' p.m., ')
        row[5] = row[5].replace('a | ', ' a.m., ')
        row[5] = row[5].replace('0p', '0 p.m.')
        row[5] = row[5].replace('5p', '5 p.m.')
        row[5] = row[5].replace(':00', '')
        row[8] = datetime.strptime(row[8], '%Y-%m-%d').weekday()
        movieday = weekDays[row[8]]
        movie_stuff = f"\"{row[2]}\", {row[3]}, {row[6]}, " + movieday + f" at {row[5]}"
        theater_stuff = f'{row[1]}, {row[0]}, '
        listing = theater_stuff + movie_stuff
        listings.append(listing)
    
sortByTheater = sorted(listings, key=itemgetter(1))
sortByMovie = sorted(sortByTheater, key=itemgetter(2))
print(sortByMovie)

['AMC Lake Square 12, 10401-015 US Hwy. 441 South, Leesburg, "Men in Black: International", PG-13, Standard, Saturday at 11:10 a.m., 2 p.m., 4:45 p.m., 6:30 p.m., 7:30 p.m., 10:15 p.m.', 'AMC Lake Square 12, 10401-015 US Hwy. 441 South, Leesburg, "Rocketman", R, Standard, Friday at 12:35 p.m.', 'AMC Lake Square 12, 10401-015 US Hwy. 441 South, Leesburg, "Aladdin (2019)", PG, Standard, Saturday at 10:40 a.m., 2 p.m., 4 p.m., 7:15 p.m., 9:15 p.m.', 'AMC Lake Square 12, 10401-015 US Hwy. 441 South, Leesburg, "Child\'s Play (2019)", R, Standard, Saturday at 11:30 a.m., 1:50 p.m., 4:10 p.m., 7:45 p.m., 8:45 p.m., 9:30 p.m., 10:15 p.m.', 'AMC Lake Square 12, 10401-015 US Hwy. 441 South, Leesburg, "Toy Story 4", G, Digital 3D, Friday at 11:15 a.m., 1:15 p.m., 2 p.m., 4 p.m., 4:45 p.m., 7:30 p.m., 10:15 p.m.', 'AMC Lake Square 12, 10401-015 US Hwy. 441 South, Leesburg, "Late Night (2019)", R, Standard, Friday at 3:40 p.m.', 'AMC Lake Square 12, 10401-015 US Hwy. 441 South, Leesburg, "The Secre

In [ ]:
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()